In [5]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [6]:
from IPython.display import display, HTML

display(HTML("<style>.container { width:90% !important; }</style>"))

<IPython.core.display.Javascript object>

In [7]:
## got an error while downloading online csv data. OSX
## quick fix
## https://stackoverflow.com/questions/50236117/scraping-ssl-certificate-verify-failed-error-for-http-en-wikipedia-org

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

<IPython.core.display.Javascript object>

In [8]:
import pandas as pd
import numpy as np

# https://stackoverflow.com/questions/21137150/format-suppress-scientific-notation-from-pandas-aggregation-results
pd.set_option("display.float_format", lambda x: "%.2f" % x)

<IPython.core.display.Javascript object>

In [9]:
balance_20 = pd.read_csv(
    "https://www.registrucentras.lt/aduomenys/?byla=JAR_FA_RODIKLIAI_BLNS_2020.csv"
)
balance_21 = pd.read_csv(
    "https://www.registrucentras.lt/aduomenys/?byla=JAR_FA_RODIKLIAI_BLNS_2021.csv"
)
pl_20 = pd.read_csv(
    "https://www.registrucentras.lt/aduomenys/?byla=JAR_FA_RODIKLIAI_PLNA_2020.csv"
)
pl_21 = pd.read_csv(
    "https://www.registrucentras.lt/aduomenys/?byla=JAR_FA_RODIKLIAI_PLNA_2021.csv"
)

<IPython.core.display.Javascript object>

In [133]:
sodra_20 = pd.read_csv("data/monthly-2020.csv", sep=";")
sodra_19 = pd.read_csv("data/monthly-2019.csv", sep=";")

<IPython.core.display.Javascript object>

# 1
Issifiltruojama UAB ir MB, laikotarpis nuo metu sausio 1 dienos. Imone veikia 20 ir 21 metais, sujungiame finansinius
duomenis i viena masyva.

In [32]:
uab_mb_name_list = ["Uždaroji akcinė bendrovė", "Mažoji bendrija"]
active_company = "Teisinis stat neįregistruotas"
laikotarpis_nuo_metu_pradzios_20 = "2019-01-01"
laikotarpis_nuo_metu_pradzios_21 = "2020-01-01"
balance_cols = [
    "obj_kodas",
    "obj_pav",
    "form_pav",
    "reg_date",
    "nuosavas_kapitalas",
    "mok_sumos_ir_isipareigojimai",
    "ilgalaikis_turtas",
    "trumpalaikis_turtas",
]

pl_cols = ["obj_kodas", "pardavimo_pajamos", "grynasis_pelnas"]

<IPython.core.display.Javascript object>

In [61]:
b20 = balance_20[
    (balance_20.stat_pav.map(lambda x: x == active_company))
    & balance_20.form_pav.isin(uab_mb_name_list)
    & balance_20.laikotarpis_nuo.map(lambda x: x == laikotarpis_nuo_metu_pradzios_20)
][balance_cols].dropna()


pl20 = pl_20[
    pl_20.laikotarpis_nuo.map(lambda x: x == laikotarpis_nuo_metu_pradzios_20)
][pl_cols].dropna()
finansai20 = b20.merge(pl20, on="obj_kodas")
finansai20.sample(3)

,obj_kodas,obj_pav,form_pav,reg_date,nuosavas_kapitalas,mok_sumos_ir_isipareigojimai,ilgalaikis_turtas,trumpalaikis_turtas,pardavimo_pajamos,grynasis_pelnas
54040,303073693,"UAB ""Kosmita""",Uždaroji akcinė bendrovė,2020-04-28,15634.00,4898.00,759.00,19773.00,92541.00,2819.00
716,302551400,"UAB ""Visp""",Uždaroji akcinė bendrovė,2020-05-26,30955.00,5154.00,6155.00,28945.00,27916.00,-147.00
50853,124633684,"Uždaroji akcinė bendrovė ""DIONITA""",Uždaroji akcinė bendrovė,2020-07-24,102508.00,12786.00,7322.00,107754.00,255701.00,1046.00


<IPython.core.display.Javascript object>

In [60]:
b21 = balance_21[
    (balance_21.stat_pav.map(lambda x: x == active_company))
    & balance_21.form_pav.isin(uab_mb_name_list)
    & balance_21.laikotarpis_nuo.map(lambda x: x == laikotarpis_nuo_metu_pradzios_21)
][balance_cols].dropna()


pl21 = pl_21[
    pl_21.laikotarpis_nuo.map(lambda x: x == laikotarpis_nuo_metu_pradzios_21)
][pl_cols].dropna()
finansai21 = b21.merge(pl21, on="obj_kodas")
finansai21.sample(3)

,obj_kodas,obj_pav,form_pav,reg_date,nuosavas_kapitalas,mok_sumos_ir_isipareigojimai,ilgalaikis_turtas,trumpalaikis_turtas,pardavimo_pajamos,grynasis_pelnas
2356,303268017,"UAB ""Ukra""",Uždaroji akcinė bendrovė,2021-06-01,-5844.00,1490924.00,1233641.00,251439.00,595.00,-3061.00
31002,120994413,"Uždaroji akcinė bendrovė ""DEVORA""",Uždaroji akcinė bendrovė,2021-05-29,-4784.00,5540.00,187.00,569.00,452.00,-454.00
19302,300599581,"UAB ""KMK plius""",Uždaroji akcinė bendrovė,2021-04-15,62085.00,5236.00,14481.00,52840.00,151670.00,11974.00


<IPython.core.display.Javascript object>

In [62]:
type(finansai20.obj_kodas[0]), type(finansai21.obj_kodas[0])

(numpy.int64, numpy.int64)

<IPython.core.display.Javascript object>

In [65]:
finansai_merged = finansai20.merge(
    finansai21[
        [
            "obj_kodas",
            "nuosavas_kapitalas",
            "mok_sumos_ir_isipareigojimai",
            "ilgalaikis_turtas",
            "trumpalaikis_turtas",
            "pardavimo_pajamos",
            "grynasis_pelnas",
        ]
    ],
    on="obj_kodas",
    suffixes=["_20", "_21"],
)

<IPython.core.display.Javascript object>

In [134]:
sodra_20.columns = sodra_19.columns = [
    "sodra_kodas",
    "obj_kodas",
    "obj_pav",
    "savivaldybe",
    "veiklos_kodas",
    "veiklos_aprasymas",
    "menuo",
    "vid_atlyginimas",
    "darbuotoju_skaicius",
    "autoriniai_atlyginimai",
    "autoriniai_darbuotojai",
    "sodros_imoka",
]
sodra_columns = [
    "obj_kodas",
    "savivaldybe",
    "veiklos_kodas",
    "veiklos_aprasymas",
    "menuo",
    "vid_atlyginimas",
    "darbuotoju_skaicius",
]
sodra_meta_columns = [
    "obj_kodas",
    "savivaldybe",
    "veiklos_kodas",
    "veiklos_aprasymas",
]

<IPython.core.display.Javascript object>

In [139]:
sdr20 = sodra_20[
    sodra_20.obj_kodas.isin(finansai_merged.obj_kodas.tolist())
    & (sodra_20.menuo == 202012)
][sodra_columns].dropna()

sdr20.rename(
    columns={
        "vid_atlyginimas": "vid_atlyginimas_21",
        "darbuotoju_skaicius": "darbuotoju_skaicius_21",
    },
    inplace=True,
)

<IPython.core.display.Javascript object>

In [140]:
sdr19 = sodra_19[
    sodra_19.obj_kodas.isin(finansai_merged.obj_kodas.tolist())
    & (sodra_19.menuo == 201912)
][sodra_columns].dropna()

sdr19.rename(
    columns={
        "vid_atlyginimas": "vid_atlyginimas_20",
        "darbuotoju_skaicius": "darbuotoju_skaicius_20",
    },
    inplace=True,
)

<IPython.core.display.Javascript object>

In [146]:
sdr_df = sdr19.merge(
    sdr20[["obj_kodas", "vid_atlyginimas_21", "darbuotoju_skaicius_21"]], on="obj_kodas"
)

<IPython.core.display.Javascript object>

In [147]:
final = finansai_merged.merge(sdr_df, on="obj_kodas")

<IPython.core.display.Javascript object>

In [152]:
final.columns.tolist()

['obj_kodas',
 'obj_pav',
 'form_pav',
 'reg_date',
 'nuosavas_kapitalas_20',
 'mok_sumos_ir_isipareigojimai_20',
 'ilgalaikis_turtas_20',
 'trumpalaikis_turtas_20',
 'pardavimo_pajamos_20',
 'grynasis_pelnas_20',
 'nuosavas_kapitalas_21',
 'mok_sumos_ir_isipareigojimai_21',
 'ilgalaikis_turtas_21',
 'trumpalaikis_turtas_21',
 'pardavimo_pajamos_21',
 'grynasis_pelnas_21',
 'savivaldybe',
 'veiklos_kodas',
 'veiklos_aprasymas',
 'menuo',
 'vid_atlyginimas_20',
 'darbuotoju_skaicius_20',
 'vid_atlyginimas_21',
 'darbuotoju_skaicius_21']

<IPython.core.display.Javascript object>

In [154]:
final.sample(10).to_csv("data/3_uzduotis_data_sample.csv")

<IPython.core.display.Javascript object>